In [ ]:
import syft as sy
from syft.abstract_node import NodeType


In [ ]:
low_side = sy.Orchestra.launch(
    name="canada-1",
    node_side_type="low",
    local_db=True,
    reset=True,
#     enable_warnings=True,
)

In [ ]:
high_side = sy.Orchestra.launch(
    name="canada-2",
    local_db=True,
    reset=True,
#     enable_warnings=True,
)

In [ ]:
client_low_do = low_side.login(email="info@openmined.org", password="changethis")
client_high_do = high_side.login(email="info@openmined.org", password="changethis")

client_low_do.register(name="Sheldon", email="sheldon@caltech.edu", password="changethis", password_verify="changethis")
client_low_ds = low_side.login(email="sheldon@caltech.edu", password="changethis")

In [ ]:
import numpy as np

mock = np.random.random(10)
real = np.random.random(10)

dataset = sy.Dataset(
        name="my-dataset",
        description="abc",
        asset_list=[
            sy.Asset(
                name="numpy-data",
                mock=mock,
                data=real,
                shape=real.shape,
                mock_is_real=True,
            )
        ],
    )

client_low_do.upload_dataset(dataset)

In [ ]:
import numpy as np

mock = np.random.random(10)
real = np.random.random(10)

dataset = sy.Dataset(
        name="my-dataset",
        description="abc",
        asset_list=[
            sy.Asset(
                name="numpy-data",
                mock=mock,
                data=real,
                shape=real.shape,
                mock_is_real=True,
            )
        ],
    )

client_high_do.upload_dataset(dataset)

In [ ]:
data_low = client_low_do.datasets[0].assets[0]

@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()

client_low_ds.code.request_code_execution(compute_mean)

In [ ]:
client_low_do._fetch_api(client_low_do.credentials)

In [ ]:
low_request = client_low_do.requests[0]

In [ ]:
from syft.service.code.user_code import UserCode
from syft.service.context import AuthedServiceContext

In [ ]:
client_high_do.code.submit(low_request.code)

In [ ]:
client_high_do._fetch_api(client_high_do.credentials)

In [ ]:
data_high = client_high_do.datasets[0].assets[0]


In [ ]:
data_high

In [ ]:
list(high_side.python_node.get_service("usercodeservice").stash.partition.data.values())[0].input_policy

In [ ]:
res = client_high_do.code.compute_mean(data=data_high)

In [ ]:
res

In [ ]:
r = client_low_do.requests[0]

In [ ]:
r.accept_by_depositing_result(res)

In [ ]:
res_low = client_low_ds.code.compute_mean(data=data_low)

In [ ]:
res_low.get()

In [ ]:
# dataset.asset_list[0].data.mean(data)

In [ ]:
data_high.pointer